In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
import ast
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)

## Functions

In [185]:
def summary_stat(df, annual_factor, q=0.05):
    '''summary assets' mean return, voaltility(stdev) and sharpe ratio'''
    result = pd.DataFrame()
    result["mean"] = df.mean() * annual_factor
    result["volatility"] = df.std() * np.sqrt(annual_factor)
    result["Sharpe Ratio"] = result["mean"]/result["volatility"]

    return result

def sub_summarize(df, year_list, annualize=12):
    col_name = ["mean", "vol", "Sharpe"]
    res = pd.DataFrame(columns=col_name)
    i = 0
    for y in year_list:
        sub1 = df.loc[y[0]: y[1]]
        res.loc[i, "mean"] = sub1.mean() * annualize
        res.loc[i, "vol"] = sub1.std() * np.sqrt(annualize)
        res.loc[i, "Sharpe"] = res.loc[i, "mean"]/res.loc[i, "vol"]
        # res.loc[i, "skewness"] = sub1.skew()

        i += 1
    res.index = pd.Series([repr(y)[1:-1] for y in year_list])
    return res

In [41]:
def tailMetrics(returns, ret, quantile=.05, relative=False):
    
    # returns: excess return
    # ret: total return
    metrics = pd.DataFrame(index=ret.columns)

    VaR = returns.quantile(quantile)
    CVaR = (returns[returns < returns.quantile(quantile)]).mean()

    if relative:
        VaR /= returns.std()
        CVaR /= returns.std()

    metrics[f'VaR ({quantile})'] = VaR
    metrics[f'CVaR ({quantile})'] = CVaR

    cum_ret = (1 + ret).cumprod()
    rolling_max = cum_ret.cummax()  # cummax function
    drawdown = (cum_ret - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    metrics['Max Drawdown'] = max_drawdown

    return metrics

In [68]:
def get_capm_matrics(targets, regressors, add_constant=True, annualize_factor=12):

    result = pd.DataFrame(index=targets.columns)
    resid_matrix = pd.DataFrame(columns=targets.columns)
    t_p_value = pd.DataFrame(index=targets.columns)

    if add_constant:
        X = sm.add_constant(regressors)
    else:
        X = regressors.copy()
    for column in targets.columns:
        y = targets[[column]]
        model = sm.OLS(y, X, missing='drop').fit()
        if add_constant:
            result.loc[column, "alpha"] = model.params['const'] * \
                annualize_factor
        result.loc[column, regressors.columns] = model.params[regressors.columns]

        result.loc[column, "R-squared"] = model.rsquared

        resid_matrix[column] = model.resid
        if add_constant:
            t_p_value.loc[column, "t-value"] = model.tvalues['const']
            t_p_value.loc[column, "p-value"] = model.pvalues['const']

    return result, resid_matrix, t_p_value

In [170]:
def summary_stats_bm(series, bm, annual_fac=12):
    ss_df = pd.DataFrame(data=None)
    ss_df['Mean'] = series.mean() * annual_fac
    ss_df['Vol'] = series.std() * np.sqrt(annual_fac)
    ss_df['Sharpe (Mean/Vol)'] = ss_df['Mean'] / ss_df['Vol']

    X = sm.add_constant(bm.loc[series.index])
    alphas, betas = [], []
    for col in series.columns:
        y = series[[col]]
        reg = sm.OLS(y, X).fit().params
        alphas.append(reg[0] * annual_fac)
        betas.append(reg[1])
    ss_df["alpha"] = alphas
    ss_df["beta"] = betas

    cum_ret = (1 + series).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    ss_df['Max Drawdown'] = drawdown.min()

    return ss_df

## 2 Analyzing GMO
### 2.1 Calculate the mean, volatility, and Sharpe ratio for GMWAX.

In [35]:
ret = pd.read_excel("../data/gmo_analysis_data.xlsx", sheet_name=2, index_col=0)
rf = pd.read_excel("../data/gmo_analysis_data.xlsx", sheet_name=3, index_col=0)
GMWAX_ex = (ret["GMWAX"] - rf["US3M"]).dropna()

In [36]:
year_list = [("1996", "2011"), ("2012", "2022"),("1996", "2022")]
sub_summarize(GMWAX_ex, year_list, annualize=12)


,mean,vol,Sharpe
"'1996', '2011'",0.0158,0.125,0.1266
"'2012', '2022'",0.0366,0.092,0.3982
"'1996', '2022'",0.0245,0.1123,0.2181


### 2.2 GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. For all three samples, analyze extreme scenarios by looking at

In [66]:
ret_ex = ret.subtract(rf["US3M"], axis=0).dropna()
tailMetrics(ret_ex, ret.dropna(), quantile=.05, relative=False)

,VaR (0.05),CVaR (0.05),Max Drawdown
SPY,-0.0800,-0.1008,-0.5080
GMWAX,-0.0483,-0.0860,-0.3552


#### (a) Does GMWAX have high or low tail-risk as seen by these stats? 

* Compared to SPY, yes. It has lower VaR and CVaR, also a lower max drawdown. 

#### (b) Does that vary much across the two subsamples?

* Yes, GMO's VaR improves notably in the second subsample, and drawdown is also much lower in the second subsample. 

In [67]:
sub1_ex, sub1_ret = ret_ex[:"2011"].dropna(), ret[:"2011"].dropna()
sub2_ex, sub2_ret = ret_ex["2012": "2022"], ret["2012": "2022"]
sub1 = tailMetrics(sub1_ex, sub1_ret,
                   quantile=.05, relative=False)
sub2 = tailMetrics(sub2_ex, sub2_ret,
                   quantile=.05, relative=False)
tail1 = pd.concat([sub1, sub2])
tail1.index = ["SPY: 1996-2011", "GMWAX: 1996-2011",
               "SPY: 2012-2022", "GMWAX: 2012-2022"]
tail1

,VaR (0.05),CVaR (0.05),Max Drawdown
SPY: 1996-2011,-0.0802,-0.1051,-0.5080
GMWAX: 1996-2011,-0.0598,-0.0965,-0.3552
SPY: 2012-2022,-0.0687,-0.0905,-0.2393
GMWAX: 2012-2022,-0.0397,-0.0618,-0.2168


### 2.3 For all three samples, regress excess returns of GMWAX on excess returns of SPY.

#### (a) Report the estimated alpha, beta, and r-squared.

In [71]:
summary, _, _ = get_capm_matrics(ret_ex[["GMWAX"]], ret_ex[["SPY"]], add_constant=True, annualize_factor=12)
summary

,alpha,SPY,R-squared
GMWAX,-0.017,0.5456,0.5777


In [72]:
summary, _, _ = get_capm_matrics(
    sub1_ex[["GMWAX"]], sub1_ret[["SPY"]], add_constant=True, annualize_factor=12)
summary

,alpha,SPY,R-squared
GMWAX,-0.0209,0.5353,0.5002


In [73]:
summary, _, _ = get_capm_matrics(
    sub2_ex[["GMWAX"]], sub2_ret[["SPY"]], add_constant=True, annualize_factor=12)
summary

,alpha,SPY,R-squared
GMWAX,-0.0384,0.5628,0.7634


#### (b) Is GMWAX a low-beta strategy? Has that changed since the case?

* GMWAX beta to SPY is around 0.54 to 0.56, as a multiasset allocation strategy, we can consider it a low-beta strategy. 
* In each subsample, beta does not change much, beta varies from 0.54-0.56. 

#### (c) Does GMWAX provide alpha? Has that changed across the subsamples?

For each subsample, alpha < 0. GMWAC didn't provide alpha in either subsample. 

## 3 Forecast Regressions
### 3.1 Consider the lagged regression

**using total return**

In [97]:
factor = pd.read_excel("../data/gmo_analysis_data.xlsx", sheet_name=1, index_col=0)
factor = factor.shift().merge(ret[["SPY"]], left_index=True, right_index=True)
factor.head()

,DP,EP,US10Y,SPY
1993-02-28,NaN,NaN,NaN,0.0107
1993-03-31,2.82,4.44,6.03,0.0224
1993-04-30,2.77,4.41,6.03,-0.0256
1993-05-31,2.82,4.44,6.05,0.0270
1993-06-30,2.81,4.38,6.16,0.0037


In [84]:
summary1, _, _ = get_capm_matrics(
    factor[["SPY"]], factor[["DP"]], add_constant=True, annualize_factor=12)
summary1

,alpha,DP,R-squared
SPY,-0.1129,0.0094,0.0094


In [85]:
summary2, _, _ = get_capm_matrics(
    factor[["SPY"]], factor[["EP"]], add_constant=True, annualize_factor=12)
summary2


,alpha,EP,R-squared
SPY,-0.0712,0.0032,0.0086


In [86]:
summary3, _, _ = get_capm_matrics(
    factor[["SPY"]], factor[["DP", "EP", "US10Y"]], add_constant=True, annualize_factor=12)
summary3

,alpha,DP,EP,US10Y,R-squared
SPY,-0.1792,0.008,0.0027,-0.001,0.0163


### 3.2 For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.

In [157]:
predict1 = factor[["DP"]].dropna() * summary1.iloc[0, 1] + summary1.iloc[0, 0]/12
predict2 = factor[["EP"]].dropna() * summary2.iloc[0, 1] + summary2.iloc[0, 0]/12
predict3 = (factor[["DP", "EP", "US10Y"]].dropna() * summary3.iloc[0, 1:4]).sum(axis=1) + summary3.iloc[0, 0]/12
predict1.columns = ["SPY"]
predict2.columns = ["SPY"]
predict3 = predict3.to_frame("SPY")

In [177]:
predict_all = pd.concat([p*100*factor[["SPY"]] for p in [predict1, predict2, predict3]], axis=1)
predict_all.columns = ["DP", "EP", "3-Factor"]

In [211]:
summary_stats_bm(predict_all.iloc[1:, ], ret.iloc[1:, 0])

,Mean,Vol,Sharpe (Mean/Vol),alpha,beta,Max Drawdown
DP,0.1095,0.1490,0.7348,0.0207,0.8611,-0.6530
EP,0.1078,0.1286,0.8383,0.0322,0.7327,-0.3823
3-Factor,0.1250,0.1456,0.8588,0.0451,0.7750,-0.5221


### 3.3 GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. Let’s consider risk characteristics.

#### (a) calculate the monthly VaR for π = .05. Just use the quantile of the historic data for this VaR calculation.

In [195]:
predict_all.dropna().quantile(.05).append(ret.quantile(.05)).to_frame("VaR(0.05)")

,VaR(0.05)
DP,-0.0523
EP,-0.0541
3-Factor,-0.0642
SPY,-0.0739
GMWAX,-0.0473


#### (b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

* Dynamic portfolio has a higher mean however higher volatility compared to risk-free rate. 
* From the mean return view, dynamic portfolio outperformed risk-free rate. However, if we use Sharpe ratio as an evaluation standard, it under-performed the risk-free rate. 

In [192]:
summary_stat(predict_all.join(rf).dropna(), 12)

,mean,volatility,Sharpe Ratio
DP,0.1095,0.1490,0.7348
EP,0.1078,0.1286,0.8383
3-Factor,0.1250,0.1456,0.8588
US3M,0.0229,0.0061,3.7512


#### (c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?

In [332]:
predict_ind = predict_all.dropna().copy()
predict_ind[predict_all >= 0] = 0
predict_ind[predict_all < 0] = 1
underperform_rate = (predict_ind.sum(axis=0)/predict_ind.shape[0]*100).to_frame("Underperformed (%)")
underperform_rate

,Underperformed (%)
DP,35.1124
EP,35.6742
3-Factor,31.4607


#### (d) Do you believe the dynamic strategy takes on extra risk??

* No, judging by the tail risk metrics and volatility of the dynamic strategies compared to SPY it does not seem like these strategies take on extra risk on the whole.

## 4 Out-of-Sample Forecasting
#### 4.1 Report the out-of-sample R2

In [300]:
# model = RollingOLS.from_formula("SPY ~ DP + EP", data=factor, window=factor.shape[0], min_nobs=60, expanding=True).fit()
model = RollingOLS(
    factor[["SPY"]], sm.add_constant(factor[["DP", "EP"]]), window=factor.shape[0], min_nobs=60, expanding=True).fit()

params = model.params.shift()
params.tail()

,const,DP,EP
2022-06-30,-0.0167,0.0073,0.0026
2022-07-31,-0.0176,0.0078,0.0025
2022-08-31,-0.0172,0.0074,0.0026
2022-09-30,-0.0175,0.0076,0.0026
2022-10-31,-0.0183,0.0081,0.0024


In [301]:
predict_spy = (params[["DP", "EP"]] * factor[["DP", "EP"]]
               ).sum(axis=1) + params["const"]
predict_err = np.square(factor["SPY"].subtract(predict_spy)).sum()
predict_err

0.6184327462302773

In [302]:
zero_pre_spy = factor[["SPY"]].expanding(60).mean().shift()
zero_pre_err = np.square((factor[["SPY"]] - zero_pre_spy)).sum()

In [303]:
1 - predict_err/zero_pre_err[0]


-0.013733571597686511

###  4.2 Re-do problem 3.2 using this OOS forecast. How much better/worse is the OOS strategy compared to the in-sample version of 3.2?

* Worse. 

In [311]:
oos_port = (predict_spy * 100 * ret["SPY"]).to_frame("oos_portfolio")
summary_stats_bm(oos_port.iloc[61:, ], ret.iloc[61:, 0])

,Mean,Vol,Sharpe (Mean/Vol),alpha,beta,Max Drawdown
oos_portfolio,0.093,0.2293,0.4059,0.0549,0.4585,-0.7609


### 4.3 Re-do problem 3.3 using this OOS forecast.

* OOS forecast has higher risk than insample forecast. 

In [316]:
tail_risk = oos_port.dropna().quantile(.05).append(
    predict_all.dropna().quantile(.05)).to_frame("VaR(0.05)")
tail_risk

,VaR(0.05)
oos_portfolio,-0.0742
DP,-0.0523
EP,-0.0541
3-Factor,-0.0642


In [317]:
summary_stat(oos_port.join(predict_all).dropna(), 12)

,mean,volatility,Sharpe Ratio
oos_portfolio,0.0930,0.2293,0.4059
DP,0.0841,0.1520,0.5531
EP,0.0830,0.1301,0.6384
3-Factor,0.1056,0.1503,0.7027


In [333]:
oos_rate = ((oos_port.dropna() < 0).sum() /
            oos_port.shape[0]).to_frame("Underperformed (%)") * 100
underperform_rate.append(oos_rate)

,Underperformed (%)
DP,35.1124
EP,35.6742
3-Factor,31.4607
oos_portfolio,31.6527
